In [3]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_DIR = 'data'

# Paths
train_dir = os.path.join(DATA_DIR, 'train')
val_dir = os.path.join(DATA_DIR, 'val')
test_dir = os.path.join(DATA_DIR, 'test')

# ✅ Train data with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,              # normalize to [0, 1]
    rotation_range=20,           # random rotation
    zoom_range=0.2,              # random zoom
    horizontal_flip=True         # random horizontal flip
)

# ✅ Validation & test: only rescale
val_test_datagen = ImageDataGenerator(rescale=1./255)

# ✅ Load from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 6225 images belonging to 11 classes.
Found 1092 images belonging to 11 classes.
Found 3187 images belonging to 11 classes.


In [4]:
# ✅ Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# ✅ Confirm TensorFlow & GPU
print("TensorFlow version:", tf.__version__)
print("GPUs available:", tf.config.list_physical_devices('GPU'))

# ✅ Parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_DIR = 'data'

train_dir = os.path.join(DATA_DIR, 'train')
val_dir = os.path.join(DATA_DIR, 'val')
test_dir = os.path.join(DATA_DIR, 'test')

# ✅ Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# ✅ CNN Model Architecture
cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()

# ✅ Callbacks
callbacks = [
    EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True),
    ModelCheckpoint('cnn_fish_model.h5', save_best_only=True)
]

# ✅ Train the Model
print("🚀 Training CNN...")
history_cnn = cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=callbacks
)

# ✅ Evaluate on Test Data
test_loss, test_acc = cnn_model.evaluate(test_generator)
print(f"✅ CNN Test Accuracy: {test_acc:.4f} | Loss: {test_loss:.4f}")


TensorFlow version: 2.19.0
GPUs available: []
Found 6225 images belonging to 11 classes.
Found 1092 images belonging to 11 classes.
Found 3187 images belonging to 11 classes.


d:\Downloads\Multiclass Fish Image Classification\.conda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 11)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,170,379 (42.61 MB)

...

195/195 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.8607 - loss: 0.3859 - val_accuracy: 0.9496 - val_loss: 0.1667
Epoch 10/15
195/195 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - accuracy: 0.8719 - loss: 0.3505 - val_accuracy: 0.9396 - val_loss: 0.2234
Epoch 11/15
195/195 ━━━━━━━━━━━━━━━━━━━━ 1480s 8s/step - accuracy: 0.8636 - loss: 0.3963 - val_accuracy: 0.9148 - val_loss: 0.2702
Epoch 12/15
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8824 - loss: 0.3338

195/195 ━━━━━━━━━━━━━━━━━━━━ 269s 1s/step - accuracy: 0.8824 - loss: 0.3338 - val_accuracy: 0.9570 - val_loss: 0.1631
Epoch 13/15
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8970 - loss: 0.3022

195/195 ━━━━━━━━━━━━━━━━━━━━ 260s 1s/step - accuracy: 0.8970 - loss: 0.3022 - val_accuracy: 0.9615 - val_loss: 0.1319
Epoch 14/15
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9093 - loss: 0.2746

195/195 ━━━━━━━━━━━━━━━━━━━━ 258s 1s/step - accuracy: 0.9093 - loss: 0.2745 - val_accuracy: 0.9652 - val_loss: 0.1315
Epoch 15/15
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9074 - loss: 0.2636

195/195 ━━━━━━━━━━━━━━━━━━━━ 259s 1s/step - accuracy: 0.9074 - loss: 0.2638 - val_accuracy: 0.9789 - val_loss: 0.1105
100/100 ━━━━━━━━━━━━━━━━━━━━ 64s 641ms/step - accuracy: 0.9787 - loss: 0.0981
✅ CNN Test Accuracy: 0.9783 | Loss: 0.0932


In [5]:
cnn_model.save("cnn_fish_model.keras")


In [ ]:
import os, time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# ✅ Check TensorFlow and GPU
print("TensorFlow Version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# ✅ Parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 15
DATA_DIR = 'data'

# ✅ Folder Paths
train_dir = os.path.join(DATA_DIR, 'train')
val_dir   = os.path.join(DATA_DIR, 'val')
test_dir  = os.path.join(DATA_DIR, 'test')

# ✅ Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)

def train_model(base_model_fn, model_name):
    print(f"\n🚀 Training {model_name}...")
    start = time.time()

    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(train_generator.num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    callbacks = [
        EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True),
        ModelCheckpoint(f"{model_name.lower()}_fish_model.h5", save_best_only=True)
    ]

    history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
    )
    test_loss, test_acc = model.evaluate(test_generator)
    print(f"✅ {model_name} Test Accuracy: {test_acc:.4f} | Loss: {test_loss:.4f}")
    print(f"🕒 Time for {model_name}: {(time.time() - start)/60:.2f} mins")

    return model_name, test_acc, history

model_results = []

for base_model, name in [
    (MobileNetV2, "MobileNetV2"),
    (InceptionV3, "InceptionV3")
    
]:
    result = train_model(base_model, name)
    model_results.append(result)
    


In [ ]:
import os, time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

print("TensorFlow:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 10
DATA_DIR = 'data'

train_dir = os.path.join(DATA_DIR, 'train')
val_dir   = os.path.join(DATA_DIR, 'val')
test_dir  = os.path.join(DATA_DIR, 'test')

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
val_generator = val_test_datagen.flow_from_directory(val_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
test_generator = val_test_datagen.flow_from_directory(test_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

def train_model(base_model_fn, model_name, unfreeze_last=0):
    print(f"\n🚀 Training {model_name}...")
    start = time.time()

    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    if unfreeze_last > 0:
        base_model.trainable = True
        for layer in base_model.layers[:-unfreeze_last]:
            layer.trainable = False
    else:
        base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(train_generator.num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    callbacks = [
        EarlyStopping(patience=3, restore_best_weights=True),
        ModelCheckpoint(f"{model_name.lower()}_fish_model.h5", save_best_only=True)
    ]

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=1
    )

    test_loss, test_acc = model.evaluate(test_generator)
    print(f"✅ {model_name} Test Accuracy: {test_acc:.4f} | Loss: {test_loss:.4f}")
    print(f"🕒 Time for {model_name}: {(time.time() - start)/60:.2f} mins")
    return model_name, test_acc, history

model_results = []

# EfficientNetB0: no fine-tuning (frozen)
model_results.append(train_model(EfficientNetB0, "EfficientNetB0"))


In [2]:
import os, time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# ✅ GPU Check
print("TensorFlow:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Parameters
IMAGE_SIZE = (224, 224)  # or (160, 160) for more speed
BATCH_SIZE = 16
EPOCHS = 10
DATA_DIR = 'data'

train_dir = os.path.join(DATA_DIR, 'train')
val_dir   = os.path.join(DATA_DIR, 'val')
test_dir  = os.path.join(DATA_DIR, 'test')

# Image Generators
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)
val_generator = val_test_datagen.flow_from_directory(
    val_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)
test_generator = val_test_datagen.flow_from_directory(
    test_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)

def train_nasnet_model():
    print("\n🚀 Training NASNetMobile...")
    start = time.time()

    base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    base_model.trainable = False  # Freeze all layers

    # Add custom layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(train_generator.num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    # Callbacks
    callbacks = [
        EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True),
        ModelCheckpoint("nasnetmobile_fish_model.h5", save_best_only=True)
    ]

    # Train
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=1
    )

    # Evaluate
    test_loss, test_acc = model.evaluate(test_generator)
    print(f"✅ NASNetMobile Test Accuracy: {test_acc:.4f} | Loss: {test_loss:.4f}")
    print(f"🕒 Training Time: {(time.time() - start)/60:.2f} minutes")

    return model, history

nasnet_model, nasnet_history = train_nasnet_model()


TensorFlow: 2.19.0
GPU Available: []
Found 6225 images belonging to 11 classes.
Found 1092 images belonging to 11 classes.
Found 3187 images belonging to 11 classes.

🚀 Training NASNetMobile...
19993432/19993432 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


d:\Downloads\Multiclass Fish Image Classification\.conda\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 957ms/step - accuracy: 0.3627 - loss: 1.8959

390/390 ━━━━━━━━━━━━━━━━━━━━ 480s 1s/step - accuracy: 0.3631 - loss: 1.8947 - val_accuracy: 0.8269 - val_loss: 0.7180
Epoch 2/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 743ms/step - accuracy: 0.7785 - loss: 0.7488

390/390 ━━━━━━━━━━━━━━━━━━━━ 328s 841ms/step - accuracy: 0.7786 - loss: 0.7486 - val_accuracy: 0.8947 - val_loss: 0.4433
Epoch 3/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 692ms/step - accuracy: 0.8574 - loss: 0.4855

...

390/390 ━━━━━━━━━━━━━━━━━━━━ 325s 833ms/step - accuracy: 0.9215 - loss: 0.2720 - val_accuracy: 0.9396 - val_loss: 0.1987
Epoch 7/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 670ms/step - accuracy: 0.9154 - loss: 0.2598

390/390 ━━━━━━━━━━━━━━━━━━━━ 299s 766ms/step - accuracy: 0.9155 - loss: 0.2598 - val_accuracy: 0.9396 - val_loss: 0.1956
Epoch 8/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 663ms/step - accuracy: 0.9281 - loss: 0.2323

390/390 ━━━━━━━━━━━━━━━━━━━━ 295s 757ms/step - accuracy: 0.9281 - loss: 0.2323 - val_accuracy: 0.9469 - val_loss: 0.1743
Epoch 9/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 672ms/step - accuracy: 0.9382 - loss: 0.2028

390/390 ━━━━━━━━━━━━━━━━━━━━ 301s 770ms/step - accuracy: 0.9382 - loss: 0.2028 - val_accuracy: 0.9441 - val_loss: 0.1581
Epoch 10/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 673ms/step - accuracy: 0.9338 - loss: 0.2014

390/390 ━━━━━━━━━━━━━━━━━━━━ 300s 768ms/step - accuracy: 0.9338 - loss: 0.2014 - val_accuracy: 0.9441 - val_loss: 0.1513
200/200 ━━━━━━━━━━━━━━━━━━━━ 132s 660ms/step - accuracy: 0.9681 - loss: 0.1070
✅ NASNetMobile Test Accuracy: 0.9664 | Loss: 0.1204
🕒 Training Time: 57.45 minutes


In [6]:
from tensorflow.keras.models import load_model

# Load the already saved model
mobilenet_model = load_model("mobilenetv2_fish_model.h5")

# Save it again as best model
mobilenet_model.save("best_fish_model.h5")


In [7]:
mobilenet_model.save("best_fish_model.keras")
